-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Scheduling Efficient Structured Streaming Jobs

We'll use this notebook as a framework to launch multiple streams on shared resources.

This notebook contains partially refactored code with all the updates and additions that will allow us to schedule our pipelines and run them as new data arrives, including logic for dealing with partition deletes from our `bronze` table.

Also included is logic to assign each stream to a scheduler pool. Review the code below and then follow the instructions in the following cell to schedule a streaming job.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_bronze.png" width="60%" />

## Scheduling this Notebook

This notebook is designed to be scheduled against a jobs cluster.

### Create a New Job
0. Click the Jobs button on the left sidebar
0. Click the blue `+ Create Job` button
0. Name the job something unique but parseable, such as `ade-stream-<your_initials>`
0. Next to **Task**, click "Select Notebook" and use the file picker to select this notebook; click OK.
0. Next to **Cluster**, click "Edit".
0. Change the following settings **only** (and then click "Confirm"):
  - **Workers**: 2
  - Under **Advanced Options** in the **Spark Config**, set: `sql.shuffle.partitions 8`
0. Click "Run Now" to start your job.

![sql-shuffle](https://files.training.databricks.com/images/enb/med_data/sql-shuffle.png)

## Widgets

The `widgets` submodule includes a number of methods to allow interactive variables to be set while working with notebooks in the workspace with an interactive cluster. To learn more about this functionality, refer to the [Databricks documentation](https://docs.databricks.com/notebooks/widgets.html#widgets).

This notebook will focus on only two of these methods, emphasizing their utility when running a notebook as a job:
1. `dbutils.widgets.text` accepts a parameter name and a default value. This is the method through which external values can be passed into scheduled notebooks.
1. `dbutils.widgets.get` accepts a parameter name and retrieves the associated value from the widget with that parameter name.

Taken together, `dbutils.widgets.text` allows the passing of external values and `dbutils.widgets.get` allows those values to be referenced.

**NOTE**: To run this notebook in streaming mode, pass the value `False` to the `once` widget, or add this as a parameter to your scheduled job.

In [0]:
dbutils.widgets.text("once", "True")
once = eval(dbutils.widgets.get("once"))
print(f"Once: {once}")

## Configure Apache Spark Scheduler Pools for Efficiency

By default, all queries started in a notebook run in the same <a href="https://spark.apache.org/docs/latest/job-scheduling.html#scheduling-within-an-application" target="_blank">fair scheduling pool</a>. Therefore, jobs generated by triggers from all of the streaming queries in a notebook run one after another in first in, first out (FIFO) order. This can cause unnecessary delays in the queries, because they are not efficiently sharing the cluster resources.

In particular, resource-intensive streams can hog the available compute in a cluster, preventing smaller streams from achieving low latency. Configuring pools provides the capacity to fine tune your cluster to ensure processing time.

To enable all streaming queries to execute jobs concurrently and to share the cluster efficiently, you can set the queries to execute in separate scheduler pools. This **local property configuration** will be in the same notebook cell where we start the streaming query. For example:

**Run streaming query1 in scheduler pool1**
```
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool1")
df.writeStream.queryName("query1").format("parquet").start(path1)
```
**Run streaming query2 in scheduler pool2**

```
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool2")
df.writeStream.queryName("query2").format("delta").start(path2)
```

In [0]:
%run ../Includes/ade-setup

## Auto Optimize and Auto Compaction

We'll want to ensure that our bronze table and 3 parsed silver tables don't contain too many small files. Turning on Auto Optimize and Auto Compaction help us to avoid this problem. For more information on these settings, <a href="https://docs.databricks.com/delta/optimizations/auto-optimize.html" target="_blank">consult our documentation</a>.

In [0]:
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", True)
spark.conf.set("spark.databricks.delta.autoCompact.enabled", True)

# Bronze

In [0]:
dateLookup = spark.table("date_lookup").select("date", "week_part")
dateLookup.cache().count()

In [0]:
def process_bronze(source, table_name, checkpoint, once=False, processing_time="5 seconds"):
    schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"
    
    data_stream_writer = (spark.readStream
            .format("cloudFiles")
            .schema(schema)
            .option("maxFilesPerTrigger", 2)
            .option("cloudFiles.format", "json")
            .load(source)
            .join(F.broadcast(dateLookup), [F.to_date((F.col("timestamp")/1000).cast("timestamp")) == F.col("date")], "left")
            .writeStream
            .option("checkpointLocation", checkpoint)
            .partitionBy("topic", "week_part")
            .queryName("bronze")
         )
    
    if once == True:
        (data_stream_writer
            .trigger(once=True)
            .table(table_name)
            .awaitTermination(60)
        )
    else:
        (data_stream_writer
            .trigger(processingTime=processing_time)
            .table(table_name)
        )

In [0]:
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "bronze")
process_bronze(Paths.source30m, "bronze_dev", Paths.bronzeCheckpoint, once=once)

# Parse Silver Tables

In the next cell, we define a Python class to handle the queries that result in our `heart_rate_silver` and `workouts_silver`.

In [0]:
class Upsert:
    def __init__(self, query, update_temp="stream_updates"):
        self.query = query
        self.update_temp = update_temp 
        
    def upsertToDelta(self, microBatchDF, batch):
        microBatchDF.createOrReplaceTempView(self.update_temp)
        microBatchDF._jdf.sparkSession().sql(self.query)

In [0]:
# heart_rate_silver
def heart_rate_silver(source_table="bronze", once=False, processing_time="10 seconds"):
    
    query = """
        MERGE INTO heart_rate_silver a
        USING heart_rate_updates b
        ON a.device_id=b.device_id AND a.time=b.time
        WHEN NOT MATCHED THEN INSERT *
        """

    streamingMerge=Upsert(query, "heart_rate_updates")
    
    data_stream_writer = (spark.readStream
        .option("ignoreDeletes", True)
        .table(source_table)
        .filter("topic = 'bpm'")
        .select(F.from_json(F.col("value").cast("string"), "device_id LONG, time TIMESTAMP, heartrate DOUBLE").alias("v"))
        .select("v.*", F.when(F.col("v.heartrate") <= 0, "Negative BPM").otherwise("OK").alias("bpm_check"))
        .withWatermark("time", "30 seconds")
        .dropDuplicates(["device_id", "time"])
        .writeStream
        .foreachBatch(streamingMerge.upsertToDelta)
        .outputMode("update")
        .option("checkpointLocation", Paths.silverRecordingsCheckpoint)
        .queryName("heart_rate_silver")
    )
  
    if once == True:
        (data_stream_writer
            .trigger(once=True)
            .start()
            .awaitTermination(60)
        )
    else:
        (data_stream_writer
            .trigger(processingTime=processing_time)
            .start()
        )

In [0]:
# workouts_silver
def workouts_silver(source_table="bronze", once=False, processing_time="15 seconds"):
    
    query = """
        MERGE INTO workouts_silver a
        USING workout_updates b
        ON a.user_id=b.user_id AND a.time=b.time
        WHEN NOT MATCHED THEN INSERT *
        """

    streamingMerge=Upsert(query, "workout_updates")
    
    data_stream_writer = (spark.readStream
        .option("ignoreDeletes", True)
        .table(source_table)
        .filter("topic = 'workout'")
        .select(F.from_json(F.col("value").cast("string"), "user_id INT, workout_id INT, timestamp FLOAT, action STRING, session_id INT").alias("v"))
        .select("v.*")
        .select("user_id", "workout_id", F.col("timestamp").cast("timestamp").alias("time"), "action", "session_id")
        .withWatermark("time", "30 seconds")
        .dropDuplicates(["user_id", "time"])
        .writeStream
        .foreachBatch(streamingMerge.upsertToDelta)
        .outputMode("update")
        .option("checkpointLocation", Paths.silverWorkoutsCheckpoint)
        .queryName("workouts_silver")

    )

    if once == True:
        (data_stream_writer
            .trigger(once=True)
            .start()
            .awaitTermination(60)
        )
    else:
        (data_stream_writer
            .trigger(processingTime=processing_time)
            .start()
        )

In [0]:
# users
from pyspark.sql.window import Window

window = Window.partitionBy("alt_id").orderBy(F.col("updated").desc())

def batch_rank_upsert(microBatchDF, batchId):
    appId = "batch_rank_upsert"
    
    (microBatchDF
        .filter(F.col("update_type").isin(["new", "update"]))
        .withColumn("rank", F.rank().over(window)).filter("rank == 1").drop("rank")
        .createOrReplaceTempView("ranked_updates"))
    
    microBatchDF._jdf.sparkSession().sql("""
        MERGE INTO users u
        USING ranked_updates r
        ON u.alt_id=r.alt_id
        WHEN MATCHED AND u.updated < r.updated
          THEN UPDATE SET *
        WHEN NOT MATCHED
          THEN INSERT *
    """)

    (microBatchDF
         .filter("update_type = 'delete'")
         .select(
            "alt_id", 
            F.col("updated").alias("requested"), 
            F.date_add("updated", 30).alias("deadline"), 
            F.lit("requested").alias("status"))
        .write
        .format("delta")
        .mode("append")
        .option("txnVersion", batchId)
        .option("txnAppId", appId)
        .option("path", Paths.deleteRequests)
        .saveAsTable("delete_requests"))


def users_silver(source_table="bronze", once=False, processing_time="30 seconds"):

    schema = """
        user_id LONG, 
        update_type STRING, 
        timestamp FLOAT, 
        dob STRING, 
        sex STRING, 
        gender STRING, 
        first_name STRING, 
        last_name STRING, 
        address STRUCT<
            street_address: STRING, 
            city: STRING, 
            state: STRING, 
            zip: INT
        >"""

    salt = "BEANS"

    data_stream_writer = (spark.readStream
        .option("ignoreDeletes", True)
        .table(source_table)
        .filter("topic = 'user_info'")
        .dropDuplicates()
        .select(F.from_json(F.col("value").cast("string"), schema).alias("v")).select("v.*")
        .select(F.sha2(F.concat(F.col("user_id"), F.lit(salt)), 256).alias("alt_id"),
            F.col('timestamp').cast("timestamp").alias("updated"),
            F.to_date('dob','MM/dd/yyyy').alias('dob'),
            'sex', 'gender','first_name','last_name',
            'address.*', "update_type")
        .writeStream
        .foreachBatch(batch_rank_upsert)
        .outputMode("update")
        .option("checkpointLocation", Paths.usersCheckpointPath)
        .queryName("users")
    )
    
    if once == True:
        (data_stream_writer
            .trigger(once=True)
            .start()
            .awaitTermination(60)
        )
    else:
        (data_stream_writer
            .trigger(processingTime=processing_time)
            .start()
        )

In [0]:
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "silver_parsed")
heart_rate_silver(source_table="bronze_dev", once=once)
workouts_silver(source_table="bronze_dev", once=once)
users_silver(source_table="bronze_dev", once=once)

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>